# EEG-RSA Exploratory Analysis

This notebook provides a template for exploratory analysis of EEG data using Representational Similarity Analysis (RSA).

Based on Wilson et al. methodology for comparing imagery and perception representations.

In [ ]:
# Import required libraries
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mne

from preprocessing import load_raw_eeg, preprocess_raw, create_epochs
from features import extract_all_features
from rsa import compute_rdm, create_imagery_perception_model, visualize_rdm
from analysis import run_full_analysis

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
%matplotlib inline

## 1. Load and Preprocess Data

In [ ]:
# Load raw EEG data
data_path = '../data/raw/subject_01.bdf'  # Update with your file path

raw = load_raw_eeg(data_path)

# Plot raw data
raw.plot(duration=10, n_channels=30)

In [ ]:
# Preprocess: filter and re-reference
raw_preprocessed = preprocess_raw(raw, l_freq=0.1, h_freq=40.0, notch_freq=50.0)

# Plot power spectral density
raw_preprocessed.plot_psd(fmax=50)

## 2. Extract Epochs

In [ ]:
# Define events (update with your event codes)
events = mne.find_events(raw_preprocessed)

event_id = {
    'imagery_face': 1,
    'imagery_house': 2,
    'perception_face': 3,
    'perception_house': 4
}

# Create epochs
epochs = create_epochs(raw_preprocessed, events, event_id, 
                      tmin=-0.2, tmax=1.0, baseline=(-0.2, 0))

print(f'Created {len(epochs)} epochs')

In [ ]:
# Plot ERPs
epochs.average().plot(spatial_colors=True, time_unit='s')

## 3. Extract Features

In [ ]:
# Extract ERP and spectral features
features_dict = extract_all_features(epochs, 
                                    include_erp=True,
                                    include_spectral=True,
                                    normalize=True)

print(f'ERP features shape: {features_dict["erp"].shape}')
print(f'Spectral features shape: {features_dict["spectral"].shape}')
print(f'All features shape: {features_dict["all"].shape}')

## 4. Compute RDMs

In [ ]:
# Compute RDM for imagery condition
imagery_epochs = epochs['imagery_face', 'imagery_house']
imagery_features = extract_all_features(imagery_epochs, normalize=True)['all']

# Average features by condition
imagery_face_avg = imagery_features[epochs['imagery_face'].selection].mean(axis=0)
imagery_house_avg = imagery_features[epochs['imagery_house'].selection].mean(axis=0)
imagery_cond_features = np.vstack([imagery_face_avg, imagery_house_avg])

rdm_imagery = compute_rdm(imagery_cond_features, metric='correlation')

# Visualize
visualize_rdm(rdm_imagery, labels=['Face', 'House'], title='Imagery RDM')

In [ ]:
# Compute RDM for perception condition
perception_epochs = epochs['perception_face', 'perception_house']
perception_features = extract_all_features(perception_epochs, normalize=True)['all']

perception_face_avg = perception_features[epochs['perception_face'].selection].mean(axis=0)
perception_house_avg = perception_features[epochs['perception_house'].selection].mean(axis=0)
perception_cond_features = np.vstack([perception_face_avg, perception_house_avg])

rdm_perception = compute_rdm(perception_cond_features, metric='correlation')

visualize_rdm(rdm_perception, labels=['Face', 'House'], title='Perception RDM')

## 5. Compare Imagery and Perception

In [ ]:
from scipy.stats import spearmanr

# Correlation between imagery and perception RDMs
n = rdm_imagery.shape[0]
triu_idx = np.triu_indices(n, k=1)

imagery_vec = rdm_imagery[triu_idx]
perception_vec = rdm_perception[triu_idx]

corr, p_value = spearmanr(imagery_vec, perception_vec)

print(f'Spearman correlation: r = {corr:.3f}, p = {p_value:.4f}')

# Scatter plot
plt.figure(figsize=(8, 6))
plt.scatter(imagery_vec, perception_vec, alpha=0.6)
plt.plot([0, 1], [0, 1], 'r--', alpha=0.5)
plt.xlabel('Imagery Dissimilarity')
plt.ylabel('Perception Dissimilarity')
plt.title(f'Imagery vs Perception (r = {corr:.3f})')
plt.grid(alpha=0.3)
plt.show()

## 6. Model Comparison

In [ ]:
from rsa import create_imagery_perception_model, compare_models

# Create theoretical models
condition_types = ['imagery', 'imagery', 'perception', 'perception']
stimulus_types = ['face', 'house', 'face', 'house']

models = create_imagery_perception_model(condition_types, stimulus_types)

# Compare with data
# Concatenate imagery and perception RDMs
combined_rdm = np.block([[rdm_imagery, np.ones((2,2))],
                        [np.ones((2,2)), rdm_perception]])

results = compare_models(combined_rdm, models)

# Plot results
model_names = list(results.keys())
correlations = [results[m][0] for m in model_names]

plt.figure(figsize=(10, 6))
plt.bar(model_names, correlations)
plt.ylabel('Correlation with Data')
plt.title('Model Comparison')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 7. Save Results

In [ ]:
# Save RDMs
np.save('../data/outputs/rdm_imagery.npy', rdm_imagery)
np.save('../data/outputs/rdm_perception.npy', rdm_perception)

# Save epochs
imagery_epochs.save('../data/preprocessed/imagery_epochs-epo.fif', overwrite=True)
perception_epochs.save('../data/preprocessed/perception_epochs-epo.fif', overwrite=True)

print('Results saved!')